In [1]:
from decouple import AutoConfig
config = AutoConfig(search_path='./../.env')
import os

os.environ["AZURE_OPENAI_API_KEY"] = config('AZURE_API_KEY')
os.environ["AZURE_OPENAI_ENDPOINT"] = config('AZURE_ENDPOINT')


## Models

### Vanila Large Language Models (LLMs)

LLMs are primarily designed for generating contextually relevant text, with primary focus on generating, completing, and language understanding. These models are pre-trained on diverse corpus capturing linguistic patterns for language understanding. They are widely used for downstream tasks like translation, summarization, task/domain-specific fine-tuning. etc.

Some prominent examples:
- GPT-3
- llama, llama-2, llama-3

#### OpenAI Model (Azure endpoint)

In [2]:
from langchain_openai import AzureOpenAI

temp = 0.3
max_tokens = 1024
llm = AzureOpenAI(
    deployment_name="text-davinci-003",
    model_name="text-davinci-003",
    api_version = "2022-12-01",
    temperature=temp,
    max_tokens=max_tokens
)

In [3]:
print(llm.invoke("What is the meaning of life in 10 words?"))



Living with purpose, love, and joy.


### Chat or Instruction tuned Models

Chat or instruction models are specifically designed for following user instructions or engaging in conversation with the user. They are LLMs that are further fine-tuned with specific datasets. Their main focus is to understand the context from user queries and respond accordingly. They are widely used for question answering, chatbots, dialogoe systems, etc.

Some prominent examples:
- GPT-3.5-turbo, GPT-4
- llama-chat models
- claude-2

In langchain, a chat model is a language model that uses chat messages as inputs and returns chat messages as outputs.

##### Passing user message to model through HumanMessage

In [4]:
from langchain_core.messages import HumanMessage
message = [HumanMessage("What is the meaning of life in 10 words?")]

#### OpenAI models (Azure endpoints)

In [5]:
from langchain_openai import AzureChatOpenAI

In [6]:
chat_llm = AzureChatOpenAI(
    openai_api_version="2023-03-15-preview",
    azure_deployment="gpt-35-turbo",
    temperature=temp,
    max_tokens=max_tokens
)

`invoke()` call the chain on an input

In [7]:
print(chat_llm.invoke(message))

content='To love, learn, grow, connect, create, and find purpose.'


`stream()` stream back chunks of the response

In [8]:
for chunk in chat_llm.stream(message):
    print(chunk.content, end="", flush=True)

To love, learn, grow, connect, create, and find purpose.

In [9]:
chat_llm_gpt4 = AzureChatOpenAI(
    openai_api_version="2023-03-15-preview",
    azure_deployment="gpt-4",
    temperature=temp,
    max_tokens=max_tokens
)

In [10]:
print(chat_llm_gpt4.invoke(message))

content='Discover purpose, cultivate love, pursue growth, embrace experiences, seek happiness.'


P.S.: The LLM returns a string, while the ChatModel returns a message.

## Prompts and Prompt Templates

A **prompt** could be an instruction or a query that is passed to the llm. At times, it can also contain some more details in the form of context, input, or example.

A **prompt template** is a wrapper around user-prompt providing extra layer of information specific to model and task. With prompt template user input can become more dynamic, as it can provide a placeholder.

### PromptTemplate

`PromptTemplate` is used to create a template for a string prompt.

Important Functions:
- `PromptTemplate.from_template()` defines the template
- `PromptTemplate.format()` to format the defined template with user input.

Reference: [langchain PromptTemplate](https://python.langchain.com/docs/modules/model_io/prompts/quick_start/#prompttemplate)

In [11]:
from langchain_core.prompts import PromptTemplate

In [12]:
prompt = PromptTemplate.from_template("What is the meaning of life in less than {num_of_words} words {style}?")
print(prompt.format(num_of_words=100, style=""))

What is the meaning of life in less than 100 words ?


In [13]:
prompt

PromptTemplate(input_variables=['num_of_words', 'style'], template='What is the meaning of life in less than {num_of_words} words {style}?')

In [14]:
print(llm.invoke(prompt.format(num_of_words=10, style="")))



Living with purpose and joy.


In [15]:
print(llm.invoke(prompt.format(num_of_words=50, style="")))



The meaning of life is to find joy and purpose in the journey of life and to make a positive impact on the world.


In [16]:
print(llm.invoke(prompt.format(num_of_words=50, style="in a funny way")))



Life is a great big roller coaster ride, so buckle up and enjoy the ups and downs!


### ChatPromptTemplate

`ChatPromptTemplate`, prompt template for chat models, is a list of `ChatMessageTemplates`. Each `ChatMessageTemplate` contains instructions for how to format that `ChatMessage` - its role, and then also its content.

Important Functions:
- `ChatPromptTemplate.from_messages()` defines the chat template
- `ChatPromptTemplate.format_messages()` to format the defined template with user input.

Reference: 
- [langchain ChatPromptTemplate](https://python.langchain.com/docs/modules/model_io/prompts/quick_start/#chatprompttemplate)
- [OpenAI ChatCOmpletion](https://platform.openai.com/docs/guides/text-generation/chat-completions-api)

In [17]:
from langchain_core.prompts.chat import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("What is the meaning of life in less than {num_of_words} words {style}?")
message = prompt.format(num_of_words=50, style="in a funny way")

In [18]:
print(message)
print(type(message))

Human: What is the meaning of life in less than 50 words in a funny way?
<class 'str'>


In [19]:
template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

chat_message = chat_prompt.format_messages(input_language="English", 
                            output_language="Hindi", 
                            text="The meaning of life is to find joy and purpose in living, and to make a positive impact on the world.")

In [20]:
print(chat_message)
print(type(chat_message))
for msg in chat_message:
    print(msg)

[SystemMessage(content='You are a helpful assistant that translates English to Hindi.'), HumanMessage(content='The meaning of life is to find joy and purpose in living, and to make a positive impact on the world.')]
<class 'list'>
content='You are a helpful assistant that translates English to Hindi.'
content='The meaning of life is to find joy and purpose in living, and to make a positive impact on the world.'


In [21]:
print(chat_llm.invoke(chat_prompt.format_messages(input_language="English", 
                            output_language="Hindi", 
                            text="The meaning of life is to find joy and purpose in living, and to make a positive impact on the world.")))

content='जीवन का अर्थ जीवन में आनंद और उद्देश्य खोजना है, और दुनिया पर सकारात्मक प्रभाव डालना है।'


##### Using Placeholder

In [22]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)

human_template = "Summarise the converstion in {word_count} words."
humman_message_template = HumanMessagePromptTemplate.from_template(human_template)
print(humman_message_template)

chat_prompt = ChatPromptTemplate.from_messages(
    [humman_message_template, MessagesPlaceholder(variable_name="conversation")]
)
print(chat_prompt)

prompt=PromptTemplate(input_variables=['word_count'], template='Summarise the converstion in {word_count} words.')
input_variables=['conversation', 'word_count'] input_types={'conversation': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['word_count'], template='Summarise the converstion in {word_count} words.')), MessagesPlaceholder(variable_name='conversation')]


In [23]:
from langchain_core.messages import AIMessage, HumanMessage

human_message = HumanMessage(content="What is the meaning of life in less than 20 words?")
ai_message = AIMessage(
    content="""The meaning of life is to find joy and purpose in living, and to make a positive impact on the world."""
)

chat_message = chat_prompt.format_prompt(
    conversation=[human_message, ai_message], word_count=20,
)
print(chat_message)

messages=[HumanMessage(content='Summarise the converstion in 20 words.'), HumanMessage(content='What is the meaning of life in less than 20 words?'), AIMessage(content='The meaning of life is to find joy and purpose in living, and to make a positive impact on the world.')]


In [24]:
print(chat_llm_gpt4.invoke(chat_message))

content="Exploring purpose, finding joy, making a positive impact, and experiencing personal growth throughout life's journey."


For more examples, reference [langchain docs](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html), [langchain tutorials](https://python.langchain.com/docs/modules/model_io/prompts/quick_start/)

## Output Parser

`OutputParsers` convert the raw output of a language model into a format that can be used downstream.

#### PydanticOutputParser

In [40]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class Answer(BaseModel):
    thought: str = Field(description="answer with thought.")

parser = PydanticOutputParser(pydantic_object=Answer)

prompt = PromptTemplate(
    template="Answer the user query in less than {word_count} words.\n\n{format_instructions}\n\n{query}\n",
    input_variables=["word_count", "query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

response = chat_llm.invoke(prompt.format(word_count=20, query="What is the meaning of life?"))
print(response)

content='{"thought": "The meaning of life is subjective and varies from person to person."}'


In [41]:
print(parser.invoke(response))

thought='The meaning of life is subjective and varies from person to person.'


In [43]:
print(parser.invoke(response).thought)

The meaning of life is subjective and varies from person to person.


#### Built-In Parsers

##### JSONOutputParser

In [47]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()

prompt = PromptTemplate(
    template="""Return the response in JSON format with keys Question and Answer by answering the user query in less than {word_count} words.\n\n{format_instructions}\n\n{query}\n""",
    input_variables=["word_count", "query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

response = chat_llm.invoke(prompt.format(word_count=20, query="What is the meaning of life?"))
print(response)

content='{\n  "Question": "What is the meaning of life?",\n  "Answer": "The meaning of life is subjective and varies from person to person."\n}'


In [52]:
print(type(parser.invoke(response)))
print(parser.invoke(response))

<class 'dict'>
{'Question': 'What is the meaning of life?', 'Answer': 'The meaning of life is subjective and varies from person to person.'}


In [51]:
print(parser.invoke(response)['Question'])

What is the meaning of life?


In [53]:
print(parser.invoke(response)['Answer'])

The meaning of life is subjective and varies from person to person.
